In [1]:
import numpy as np
import pandas as pd

In [2]:
df_train = pd.read_csv('df_train.csv', parse_dates=['forecast_date', 'target_end_date'])

In [3]:
df_train.head()

,forecast_date,target,target_end_date,quantile,type,value,location,model
0,2021-01-31,4 wk ahead inc death,2021-02-27,0.010,quantile,13527.57889,US,BPagano-RtDriven
1,2021-01-31,4 wk ahead inc death,2021-02-27,0.025,quantile,14229.39794,US,BPagano-RtDriven
2,2021-01-31,4 wk ahead inc death,2021-02-27,0.050,quantile,14856.78719,US,BPagano-RtDriven
3,2021-01-31,4 wk ahead inc death,2021-02-27,0.100,quantile,15591.43945,US,BPagano-RtDriven
4,2021-01-31,4 wk ahead inc death,2021-02-27,0.150,quantile,16090.33317,US,BPagano-RtDriven


In [4]:
df_train = df_train[df_train.target == '1 wk ahead cum death']

In [5]:
def add_truth(df):
    target = 'cumulative_deaths'
    
    as_of = max(df_train.target_end_date) + pd.Timedelta('2 days')
    as_of = str(as_of.date())
    
    truth = pd.read_csv('data/JHU/' + target + '/truth_jhu_' + target + '_' + as_of + '.csv', parse_dates=['date'])
    truth.rename(columns={'value': 'truth'}, inplace=True)
    
    df = df.merge(truth, left_on=['target_end_date', 'location'], right_on=['date', 'location'], how='left')
    
    return(df)

In [6]:
df_train = add_truth(df_train)

In [18]:
as_of = max(df_train.target_end_date) + pd.Timedelta('2 days')
as_of = str(as_of.date())

In [19]:
truth = pd.read_csv('data/JHU/' + target + '/truth_jhu_' + target + '_' + as_of + '.csv', parse_dates=['date'])

NameError: name 'target' is not defined

In [100]:
truth.rename(columns={'value': 'truth'}, inplace=True)

In [109]:
df_train = df_train.merge(truth, left_on=['target_end_date', 'location'], right_on=['date', 'location'], how='left')

In [7]:
qs = df_train['quantile'].unique()

qs.sort()

In [103]:
qs[qs < 0.5]

array([0.01 , 0.025, 0.05 , 0.1  , 0.15 , 0.2  , 0.25 , 0.3  , 0.35 ,
       0.4  , 0.45 ])

In [104]:
# l = np.sort(qs[qs < 0.5])
# u = np.sort(qs[qs > 0.5])[::-1]

In [105]:
for i, l in enumerate(qs[qs < 0.5]):
    u = qs[-(i+1)]
    coverage = np.round(u - l, 2)
    print(coverage)

0.98
0.95
0.9
0.8
0.7
0.6
0.5
0.4
0.3
0.2
0.1


In [111]:
df_train.head()

,forecast_date,target,target_end_date,quantile,type,value,location,model,date,truth
0,2021-02-21,1 wk ahead cum death,2021-02-27,0.010,quantile,506397.38222,US,BPagano-RtDriven,2021-02-27,512938
1,2021-02-21,1 wk ahead cum death,2021-02-27,0.025,quantile,506934.75989,US,BPagano-RtDriven,2021-02-27,512938
2,2021-02-21,1 wk ahead cum death,2021-02-27,0.050,quantile,507415.14721,US,BPagano-RtDriven,2021-02-27,512938
3,2021-02-21,1 wk ahead cum death,2021-02-27,0.100,quantile,507977.66502,US,BPagano-RtDriven,2021-02-27,512938
4,2021-02-21,1 wk ahead cum death,2021-02-27,0.150,quantile,508359.66426,US,BPagano-RtDriven,2021-02-27,512938


In [115]:
d = df_train.pivot(index=df_train.columns.drop(['quantile', 'value']), columns='quantile', values='value').add_prefix('value_').reset_index()
d.columns.name = None

In [114]:
df_train.tail()

,forecast_date,target,target_end_date,quantile,type,value,location,model,date,truth
131371,2021-03-15,1 wk ahead cum death,2021-03-20,0.850,quantile,544240.0,US,epiforecasts-ensemble1,2021-03-20,541927
131372,2021-03-15,1 wk ahead cum death,2021-03-20,0.900,quantile,544472.0,US,epiforecasts-ensemble1,2021-03-20,541927
131373,2021-03-15,1 wk ahead cum death,2021-03-20,0.950,quantile,544871.0,US,epiforecasts-ensemble1,2021-03-20,541927
131374,2021-03-15,1 wk ahead cum death,2021-03-20,0.975,quantile,545485.0,US,epiforecasts-ensemble1,2021-03-20,541927
131375,2021-03-15,1 wk ahead cum death,2021-03-20,0.990,quantile,545944.0,US,epiforecasts-ensemble1,2021-03-20,541927


In [131]:
for i, l in enumerate(qs[qs < 0.5]):
    u = qs[-(i+1)]
    coverage = np.round(u - l, 2)
    
    d['wgt_iw_' + str(coverage)] = d['value_' + str(u)] - d['value_' + str(l)]
    
    d['wgt_pen_u_' + str(coverage)] = (d.truth - d['value_' + str(u)]).clip(lower=0)
    d['wgt_pen_l_' + str(coverage)] = (d['value_' + str(l)] - d.truth).clip(lower=0)
    
d['wgt_pen_u_0'] = (d.truth - d['value_0.5']).clip(lower=0)/2
d['wgt_pen_l_0'] = (d['value_0.5'] - d.truth).clip(lower=0)/2

d['wgt_iw'] = d.loc[:, d.columns.str.contains('wgt_iw')].sum(axis=1)/(len(qs)/2)
d['wgt_pen_u'] = d.loc[:, d.columns.str.contains('wgt_pen_u')].sum(axis=1)/(len(qs)/2)
d['wgt_pen_l'] = d.loc[:, d.columns.str.contains('wgt_pen_l')].sum(axis=1)/(len(qs)/2)
d['wis'] = d.wgt_iw + d.wgt_pen_u + d.wgt_pen_l

In [147]:
d.wis

0        511.748023
1         23.300485
2        433.075457
3         81.520656
4       1779.047253
           ...     
5707    1453.003803
5708    2765.931107
5709    2172.970149
5710    9243.687222
5711    2706.165612
Name: wis, Length: 5712, dtype: float64

In [161]:
d = d.loc[:, [not s[-1].isdigit() for s in d.columns]]

In [162]:
d.head()

,forecast_date,target,target_end_date,type,location,model,date,truth,wgt_iw,wgt_pen_u,wgt_pen_l,wis
0,2021-02-18,1 wk ahead cum death,2021-02-27,quantile,01,LANL-GrowthRate,2021-02-27,9930,489.248023,22.500000,0.000000,511.748023
1,2021-02-18,1 wk ahead cum death,2021-02-27,quantile,02,LANL-GrowthRate,2021-02-27,290,20.795759,0.000000,2.504726,23.300485
2,2021-02-18,1 wk ahead cum death,2021-02-27,quantile,04,LANL-GrowthRate,2021-02-27,15967,339.738973,93.336484,0.000000,433.075457
3,2021-02-18,1 wk ahead cum death,2021-02-27,quantile,05,LANL-GrowthRate,2021-02-27,5417,80.717253,0.803403,0.000000,81.520656
4,2021-02-18,1 wk ahead cum death,2021-02-27,quantile,06,LANL-GrowthRate,2021-02-27,51953,524.072773,1254.974480,0.000000,1779.047253


In [153]:
d.columns

Index(['forecast_date', 'target', 'target_end_date', 'type', 'location',
       'model', 'date', 'truth', 'value_0.01', 'value_0.025', 'value_0.05',
       'value_0.1', 'value_0.15', 'value_0.2', 'value_0.25', 'value_0.3',
       'value_0.35', 'value_0.4', 'value_0.45', 'value_0.5', 'value_0.55',
       'value_0.6', 'value_0.65', 'value_0.7', 'value_0.75', 'value_0.8',
       'value_0.85', 'value_0.9', 'value_0.95', 'value_0.975', 'value_0.99',
       'wgt_pen_u_0.98', 'wgt_pen_l_0.98', 'wgt_pen_u_0.95', 'wgt_pen_l_0.95',
       'wgt_pen_u_0.9', 'wgt_pen_l_0.9', 'wgt_pen_u_0.8', 'wgt_pen_l_0.8',
       'wgt_pen_u_0.7', 'wgt_pen_l_0.7', 'wgt_pen_u_0.6', 'wgt_pen_l_0.6',
       'wgt_pen_u_0.5', 'wgt_pen_l_0.5', 'wgt_pen_u_0.4', 'wgt_pen_l_0.4',
       'wgt_pen_u_0.3', 'wgt_pen_l_0.3', 'wgt_pen_u_0.2', 'wgt_pen_l_0.2',
       'wgt_pen_u_0.1', 'wgt_pen_l_0.1', 'wgt_pen_u_0', 'wgt_pen_l_0',
       'wgt_iw_0.98', 'wgt_iw_0.95', 'wgt_iw_0.9', 'wgt_iw_0.8', 'wgt_iw_0.7',
       'wgt_iw_0.6', '

In [150]:
d.loc[:, d.columns.str[-1].isdigit()]

AttributeError: 'Index' object has no attribute 'isdigit'

In [117]:
d.columns

Index(['forecast_date', 'target', 'target_end_date', 'type', 'location',
       'model', 'date', 'truth', 'value_0.01', 'value_0.025', 'value_0.05',
       'value_0.1', 'value_0.15', 'value_0.2', 'value_0.25', 'value_0.3',
       'value_0.35', 'value_0.4', 'value_0.45', 'value_0.5', 'value_0.55',
       'value_0.6', 'value_0.65', 'value_0.7', 'value_0.75', 'value_0.8',
       'value_0.85', 'value_0.9', 'value_0.95', 'value_0.975', 'value_0.99'],
      dtype='object')

In [7]:
def wis(df):
    qs = df['quantile'].unique()
    qs.sort()
    
    df_temp = df.pivot(index=df.columns.drop(['quantile', 'value']), 
                   columns='quantile', values='value').add_prefix('value_').reset_index()
    df_temp.columns.name = None
    
    for i, l in enumerate(qs[qs < 0.5]):
        u = qs[-(i+1)]
        coverage = np.round(u - l, 2)

        df_temp['wgt_iw_' + str(coverage)] = (1 - coverage)/2 * (df_temp['value_' + str(u)] - df_temp['value_' + str(l)])

        df_temp['wgt_pen_u_' + str(coverage)] = (df_temp.truth - df_temp['value_' + str(u)]).clip(lower=0)
        df_temp['wgt_pen_l_' + str(coverage)] = (df_temp['value_' + str(l)] - df_temp.truth).clip(lower=0)
    
    df_temp['wgt_pen_u_0'] = (df_temp.truth - df_temp['value_0.5']).clip(lower=0)/2
    df_temp['wgt_pen_l_0'] = (df_temp['value_0.5'] - df_temp.truth).clip(lower=0)/2

    df_temp['wgt_iw'] = df_temp.loc[:, df_temp.columns.str.contains('wgt_iw')].sum(axis=1)/(len(qs)/2)
    df_temp['wgt_pen_u'] = df_temp.loc[:, df_temp.columns.str.contains('wgt_pen_u')].sum(axis=1)/(len(qs)/2)
    df_temp['wgt_pen_l'] = df_temp.loc[:, df_temp.columns.str.contains('wgt_pen_l')].sum(axis=1)/(len(qs)/2)
    df_temp['wis'] = df_temp.wgt_iw + df_temp.wgt_pen_u + df_temp.wgt_pen_l
    
    df_temp = df_temp.loc[:, [not s[-1].isdigit() for s in df_temp.columns]]
    
    return df_temp

In [8]:
scores = wis(df_train)

In [9]:
scores

,forecast_date,target,target_end_date,type,location,model,date,truth,wgt_iw,wgt_pen_u,wgt_pen_l,wis
0,2021-02-18,1 wk ahead cum death,2021-02-27,quantile,01,LANL-GrowthRate,2021-02-27,9930,48.529437,20.700000,0.000000,69.229437
1,2021-02-18,1 wk ahead cum death,2021-02-27,quantile,02,LANL-GrowthRate,2021-02-27,290,2.136087,0.000000,2.304348,4.440435
2,2021-02-18,1 wk ahead cum death,2021-02-27,quantile,04,LANL-GrowthRate,2021-02-27,15967,28.967698,85.869565,0.000000,114.837263
3,2021-02-18,1 wk ahead cum death,2021-02-27,quantile,05,LANL-GrowthRate,2021-02-27,5417,6.624039,0.739130,0.000000,7.363170
4,2021-02-18,1 wk ahead cum death,2021-02-27,quantile,06,LANL-GrowthRate,2021-02-27,51953,49.122515,1154.576522,0.000000,1203.699037
...,...,...,...,...,...,...,...,...,...,...,...,...
5707,2021-03-15,1 wk ahead cum death,2021-03-20,quantile,US,PSI-DRAFT,2021-03-20,541927,136.644767,72.317391,0.000000,208.962159
5708,2021-03-15,1 wk ahead cum death,2021-03-20,quantile,US,RobertWalraven-ESG,2021-03-20,541927,276.401626,14.936522,0.000000,291.338148
5709,2021-03-15,1 wk ahead cum death,2021-03-20,quantile,US,SteveMcConnell-CovidComplete,2021-03-20,541927,198.546087,4.739130,0.000000,203.285217
5710,2021-03-15,1 wk ahead cum death,2021-03-20,quantile,US,UCSD_NEU-DeepGLEAM,2021-03-20,541927,947.933302,0.000000,145.039451,1092.972753


In [33]:
df_train.to_csv('test_wis.csv', index=False)

# Ensemble Methods

In [10]:
import sklearn
from sklearn.base import BaseEstimator
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

In [11]:
df_test = pd.read_csv('df_test.csv', parse_dates=['forecast_date', 'target_end_date'])

In [12]:
df_test.head()

,forecast_date,target,target_end_date,quantile,type,value,location,model
0,2021-03-22,1 wk ahead cum death,2021-03-27,0.025,quantile,548085.195042,US,AIpert-pwllnod
1,2021-03-22,1 wk ahead cum death,2021-03-27,0.250,quantile,549332.566250,US,AIpert-pwllnod
2,2021-03-22,1 wk ahead cum death,2021-03-27,0.750,quantile,550945.118828,US,AIpert-pwllnod
3,2021-03-22,1 wk ahead cum death,2021-03-27,0.975,quantile,552771.014775,US,AIpert-pwllnod
4,2021-03-22,1 wk ahead cum death,2021-03-27,NaN,point,549987.065698,US,AIpert-pwllnod


In [13]:
df_test = df_test[df_test.target == '1 wk ahead cum death']

In [14]:
df_test.head()

,forecast_date,target,target_end_date,quantile,type,value,location,model
0,2021-03-22,1 wk ahead cum death,2021-03-27,0.025,quantile,548085.195042,US,AIpert-pwllnod
1,2021-03-22,1 wk ahead cum death,2021-03-27,0.250,quantile,549332.566250,US,AIpert-pwllnod
2,2021-03-22,1 wk ahead cum death,2021-03-27,0.750,quantile,550945.118828,US,AIpert-pwllnod
3,2021-03-22,1 wk ahead cum death,2021-03-27,0.975,quantile,552771.014775,US,AIpert-pwllnod
4,2021-03-22,1 wk ahead cum death,2021-03-27,NaN,point,549987.065698,US,AIpert-pwllnod


In [63]:
class EWA(BaseEstimator):

    def fit(self, X, y):

        # Check that X and y have correct shape
        #X, y = check_X_y(X, y, dtype=None)

        self.X_ = X
        self.y_ = y
        
        self.n_models_ = self.X_.model.nunique()
        self.coef_ = 1/self.n_models_
        # Return the classifier
        return self

    def predict(self, X):

        # Check is fit had been called
        check_is_fitted(self)

        # Input validation
        #X = check_array(X, dtype=None)

        y_pred = 
        return self.y_[closest]

In [79]:
X = df_train.drop(columns = 'truth')

In [76]:
df_train.drop(columns='date', inplace=True)

In [78]:
X.columns.drop(['value'])

Index(['forecast_date', 'target', 'target_end_date', 'quantile', 'type',
       'location', 'model', 'date'],
      dtype='object')

In [82]:
X.set_index(['forecast_date', 'target', 'target_end_date', 'quantile', 'type',
       'location', 'model'])

value
forecast_date target               target_end_date quantile type     location model                               
2021-02-21    1 wk ahead cum death 2021-02-27      0.010    quantile US       BPagano-RtDriven        506397.38222
                                                   0.025    quantile US       BPagano-RtDriven        506934.75989
                                                   0.050    quantile US       BPagano-RtDriven        507415.14721
                                                   0.100    quantile US       BPagano-RtDriven        507977.66502
                                                   0.150    quantile US       BPagano-RtDriven        508359.66426
...                                                                                                            ...
2021-03-15    1 wk ahead cum death 2021-03-20      0.850    quantile US       epiforecasts-ensemble1  544240.00000
                                                   0.900    quantile US       epiforecasts-ensemble1  544472.00000
                                                   0.950    quantile US       epiforecasts-ensemble1  544871.00000
                                                   0.975    quantile US       epiforecasts-ensemble1  545485.00000
                                                   0.990    quantile US       epiforecasts-ensemble1  545944.00000

[131376 rows x 1 columns]

In [64]:
a = EWA().fit(df_train.drop(columns='truth'), df_train.truth)

In [66]:
a.n_models_

27

In [67]:
a.coef_

0.037037037037037035